In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the datasets
taxonomy = pd.read_csv('Taxonomy.csv')
design_within_reach = pd.read_csv('Design Within Reach.csv')
discount_school_supply = pd.read_csv('Discount_School_Supply.csv')

In [26]:
def preprocess_text(text):
    if isinstance(text, str):
        # Lowercase the text
        text = text.lower()
        # Remove special characters
        text = ''.join(e for e in text if e.isalnum() or e.isspace())
        return text
    else:
        return ''

In [27]:
# Apply preprocessing to all necessary columns in Taxonomy.csv
taxonomy['productType'] = taxonomy['productType'].apply(preprocess_text)
taxonomy['category'] = taxonomy['category'].apply(preprocess_text)
taxonomy['subCategory'] = taxonomy['subCategory'].apply(preprocess_text)

# Apply preprocessing to all necessary columns in the retailer datasets
for dataset in [design_within_reach, discount_school_supply]:
    dataset['src_pt'] = dataset['src_pt'].apply(preprocess_text)
    dataset['src_cat'] = dataset['src_cat'].apply(preprocess_text)
    dataset['src_sc'] = dataset['src_sc'].apply(preprocess_text)

# Combine productType, category, and subCategory into a single text field for Taxonomy
taxonomy['combined'] = taxonomy['productType'] + ' ' + taxonomy['category'] + ' ' + taxonomy['subCategory']

In [31]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the combined text from Taxonomy
taxonomy_tfidf = tfidf_vectorizer.fit_transform(taxonomy['combined'])

def map_categories(retailer_df, taxonomy_df, vectorizer, taxonomy_tfidf):
    # Combine the retailer's categories into a single field
    retailer_df['combined'] = retailer_df['src_pt'] + ' ' + retailer_df['src_cat'] + ' ' + retailer_df['src_sc']
    
    # Transform the retailer's combined text
    retailer_tfidf = vectorizer.transform(retailer_df['combined'])
    
    # Compute cosine similarity between retailer's data and Taxonomy
    similarities = cosine_similarity(retailer_tfidf, taxonomy_tfidf)
    
    # Find the best match for each entry in the retailer's dataset
    best_matches = similarities.argmax(axis=1)
    
    # Assign the best matches to the corresponding columns
    retailer_df['ent_pt_2'] = taxonomy_df.iloc[best_matches]['productType'].values
    retailer_df['ent_cat_2'] = taxonomy_df.iloc[best_matches]['category'].values
    retailer_df['ent_sc_2'] = taxonomy_df.iloc[best_matches]['subCategory'].values

    return retailer_df

In [34]:
# Apply the mapping function to both datasets
design_within_reach_mapped = map_categories(design_within_reach, taxonomy, tfidf_vectorizer, taxonomy_tfidf)
discount_school_supply_mapped = map_categories(discount_school_supply, taxonomy, tfidf_vectorizer, taxonomy_tfidf)

# Save the mapped datasets
design_within_reach_mapped.to_csv('Design_Within_Reach_Mapped.csv', index=False)
discount_school_supply_mapped.to_csv('Discount_School_Supply_Mapped.csv', index=False)

print("Mapping complete. Files saved.")

Mapping complete. Files saved.
